In [1]:
! pip install transformers==4.25.1 TorchCRF==1.1.0 fugashi ipadic mecab-python3 unidic-lite

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 71.2 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 613.3/613.3 kB 66.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 35.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 581.5/581.5 kB 6.5 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 12.0 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 kB 2.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 45.9 MB/s eta 0:00:00:00:01
  Created wheel for ipadic: filename=ipadic-1.0.0-py3-none-any.whl size=13556703 sha256=2c37c937e734091cfabecfdf1bd5b232f1490432271a3115586e501bca22fed7
  Stored in directory: /home/ec2-user/.cache/pip/wheels/4a/fb/28/21aa4b

In [2]:
import torch
import pandas as pd
from transformers import BertJapaneseTokenizer

In [2]:
class BERT_CRF_NER(torch.nn.Module):
    def __init__(self):
        super(BERT_CRF_NER, self).__init__()
        self.bert =BertModel.from_pretrained()
        self.classifier = torch.nn.Linear()
        self.crf = CRF()
        
    def forward(self,input_ids, attention_mask ,token_type_ids=None, label=None):
       
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        logits = self.classifier(outputs[0]) #(batch_size, sequence_size, num_labels)
        pred_labels=self.crf.viterbi_decode(logits[:,1:,:],attention_mask[:,1:])
      
        return pred_labels 

In [18]:
class Inference():
    def __init__(self, model_path,tokenizer_path):
        self.model_path = model_path
        self.tokenizer_path=tokenizer_path
        self.device = self.init_device()
        self.tokenizer = self.init_tokenizer()
        self.model = self.init_model()

    def init_device(self):
        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        return device        
    
    def init_tokenizer(self):
        tokenizer = BertJapaneseTokenizer.from_pretrained(self.tokenizer_path)
        return tokenizer
    
    def init_model(self):
        model = torch.load(self.model_path,map_location=self.device).to(self.device)
        model.eval()
        return model
    
    def inference(self,texts):
        inputs = self.tokenizer(texts, return_tensors='pt', truncation=True, padding=True,max_length=128).to(self.device)
        tokens=[self.tokenizer.tokenize(t) for t in texts]
        with torch.no_grad():
            pred_labels=self.model(inputs['input_ids'],inputs['attention_mask'])
            pred_labels=[p[:-1] for p in pred_labels]
            pred_tokens=[]
            for i in range(len(tokens)):
                pred_token=[ tokens[i][j] for j in range(len(pred_labels[i])) if pred_labels[i][j]==1 or pred_labels[i][j]==2 ]      
                pred_tokens.append(pred_token)
       
        return pred_tokens, pred_labels,tokens

In [4]:
model_path='results/run_2023.2.24_73831/model/model.pt'
tokenizer_path='cl-tohoku/bert-large-japanese'
inference=Inference(model_path=model_path,tokenizer_path=tokenizer_path)

texts=['胸水は徐々に増加していき、呼吸困難あり。胸腔穿刺ドレナージ(1600ML)実施。',
       '放射線肺臓炎',
       '胃腸炎を併発しており',
       '嘔気と嘔吐を主訴に緊急受診']
pred_tokens, pred_labels,tokens=inference.inference(texts)
pred_tokens

[['胸', '##水', '呼吸', '困難', '胸'],
 ['放射', '線', '肺', '##臓', '炎'],
 ['胃', '##腸', '炎'],
 ['嘔', '##気', '嘔', '##吐']]

In [30]:
def get_result_df(x):
    df=pd.DataFrame({
        'token': [s.replace('##', '') for s in x['tokens']],
        'preds':[1 if x['preds'][i]==2 else x['preds'][i] for i in range(len(x['preds']))],
        'th': [i== 1 or i==2 for i in x['preds']]})
    return df

def join_words(df):
    df['group'] = df['th'].diff(1).cumsum().fillna(0)
    df_g = df[df['th']].groupby('group').agg({
        'token':lambda x: ''.join(x.to_list())
        })
    df_g = df_g.drop_duplicates('token')
    elminated_tokens = ['。', '、', '(', ')', 'なし', 'いる', ':', '-']
    df_g = df_g[~df_g['token'].isin(elminated_tokens)]
    return df_g        

def result_df(texts,tokens,pred_labels):    
    result_df=pd.DataFrame({'row':texts,'tokens':tokens,'preds':pred_labels})
    #空白行対応
    result_df['row']=result_df['row'].apply(lambda x: 'o' if len(x)==0 else x)
    result_df['tokens']=result_df['tokens'].apply(lambda x: ['o'] if len(x)==0 else x)
    result_df['preds']=result_df['preds'].apply(lambda x: [0] if len(x)==0 else x)
    #抽出結果
    result_df['df']=result_df.apply(lambda x:get_result_df(x),axis=1 )                
    result_df['df_pred_pick'] = result_df['df'].apply(lambda x: join_words(x)) 
    result_df['pred_pick'] = result_df['df_pred_pick'].apply(lambda x: x['token'].to_list())
    result_df['pred_pick'] = result_df['pred_pick'].apply(lambda x: x if len(x)>0 else ['検出なし'])
    result_df['pred_pick'] = result_df['pred_pick'].apply(lambda x: [i for i in x if len(i)!=1 ])
    result_df=result_df.drop(columns=['df','df_pred_pick'])    
    return result_df

In [31]:
model_path='results/run_2023.2.24_73831/model/model.pt'
tokenizer_path='cl-tohoku/bert-large-japanese'
inference=Inference(model_path=model_path,tokenizer_path=tokenizer_path)

texts=['胸水は徐々に増加していき、呼吸困難あり。胸腔穿刺ドレナージ(1600ML)実施。',
       '放射線肺臓炎',
       '胃腸炎を併発しており',
       '嘔気と嘔吐を主訴に緊急受診']
pred_tokens, pred_labels,tokens=inference.inference(texts)
result_df=result_df(texts,tokens,pred_labels)
result_df[['row','pred_pick']]

,row,pred_pick
0,胸水は徐々に増加していき、呼吸困難あり。胸腔穿刺ドレナージ(1600ML)実施。,"[胸水, 呼吸困難]"
1,放射線肺臓炎,[放射線肺臓炎]
2,胃腸炎を併発しており,[胃腸炎]
3,嘔気と嘔吐を主訴に緊急受診,"[嘔気, 嘔吐]"


In [10]:
class BERT_CRF_NER(torch.nn.Module):
    def __init__(self):
        super(BERT_CRF_NER, self).__init__()
        self.bert =BertModel.from_pretrained()
        self.classifier = torch.nn.Linear()
        self.crf = CRF()
        
    def forward(self,input_ids, attention_mask ,token_type_ids=None, label=None):
       
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        logits = self.classifier(outputs[0]) #(batch_size, sequence_size, num_labels)
        pred_labels=self.crf.viterbi_decode(logits[:,1:,:],attention_mask[:,1:])
      
        return pred_labels 

class Inference():
    def __init__(self, model_path,tokenizer_path):
        self.model_path = model_path
        self.tokenizer_path=tokenizer_path
        self.device = self.init_device()
        self.tokenizer = self.init_tokenizer()
        self.model = self.init_model()
    
    def init_device(self):
        device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        return device        
    
    def init_tokenizer(self):
        tokenizer = BertJapaneseTokenizer.from_pretrained(self.tokenizer_path)
        return tokenizer
    
    def init_model(self):
        model = torch.load(self.model_path,map_location=self.device).to(self.device)
        model.eval()
        return model
    
    def inference(self,texts):
        inputs = self.tokenizer(texts, return_tensors='pt', truncation=True, padding=True,max_length=128).to(self.device)
        tokens=[self.tokenizer.tokenize(t) for t in texts]
        with torch.no_grad():
            pred_labels=self.model(inputs['input_ids'],inputs['attention_mask'])
            pred_labels=[p[:-1] for p in pred_labels]
            pred_tokens=[]
            for i in range(len(tokens)):
                pred_token=[ tokens[i][j] for j in range(len(pred_labels[i])) if pred_labels[i][j]==1 or pred_labels[i][j]==2 ]      
                pred_tokens.append(pred_token)
       
        return pred_labels,tokens

    def pred_pick(self,texts):    
        
        def get_result_df(x):
            df=pd.DataFrame({
                'token': [s.replace('##', '') for s in x['tokens']],
                'preds':[1 if x['preds'][i]==2 else x['preds'][i] for i in range(len(x['preds']))],
                'th': [i== 1 or i==2 for i in x['preds']]})
            return df

        def join_words(df):
            df['group'] = df['th'].diff(1).cumsum().fillna(0)
            df_g = df[df['th']].groupby('group').agg({
                'token':lambda x: ''.join(x.to_list())
                })
            df_g = df_g.drop_duplicates('token')
            elminated_tokens = ['。', '、', '(', ')', 'なし', 'いる', ':', '-']
            df_g = df_g[~df_g['token'].isin(elminated_tokens)]
            return df_g
    
        pred_labels,tokens=self.inference(texts)
        result_df=pd.DataFrame({'row':texts,'tokens':tokens,'preds':pred_labels})
        #空白行対応
        result_df['tokens']=result_df['tokens'].apply(lambda x: ['o'] if len(x)==0 else x)
        result_df['preds']=result_df['preds'].apply(lambda x: [0] if len(x)==0 else x)
        #抽出結果
        result_df['df']=result_df.apply(lambda x:get_result_df(x),axis=1 )                
        result_df['df_pred_pick'] = result_df['df'].apply(lambda x: join_words(x)) 
        result_df['pred_pick'] = result_df['df_pred_pick'].apply(lambda x: x['token'].to_list())
        result_df['pred_pick'] = result_df['pred_pick'].apply(lambda x: x if len(x)>0 else [])
        result_df['pred_pick'] = result_df['pred_pick'].apply(lambda x: [i for i in x if len(i)!=1 ])
        result_df=result_df.drop(columns=['df','df_pred_pick'])    
        
        return result_df[['row','pred_pick']]
    

In [11]:
model_path='results/run_2023.2.24_73831/model/model.pt'
tokenizer_path='cl-tohoku/bert-large-japanese'
inference=Inference(model_path=model_path,tokenizer_path=tokenizer_path)

texts=['胸水は徐々に増加していき、呼吸困難あり。胸腔穿刺ドレナージ(1600ML)実施。',
       '放射線肺臓炎',
       '胃腸炎を併発しており',
       '嘔気と嘔吐を主訴に緊急受診']

inference.pred_pick(texts)

,row,pred_pick
0,胸水は徐々に増加していき、呼吸困難あり。胸腔穿刺ドレナージ(1600ML)実施。,"[胸水, 呼吸困難]"
1,放射線肺臓炎,[放射線肺臓炎]
2,胃腸炎を併発しており,[胃腸炎]
3,嘔気と嘔吐を主訴に緊急受診,"[嘔気, 嘔吐]"


**gpu実行速度**

In [12]:
import time

In [17]:
# gpu
tic = time.time()
inference.pred_pick(['胸水は徐々に増加していき、呼吸困難あり。胸腔穿刺ドレナージ(1600ML)実施。']*100)
reftime = time.time() - tic
print(reftime)

0.8241205215454102


In [18]:
sample_text ="""

2014/06/13
Bp＋Ax（乳房部分切除＋腋窩リンパ節郭清）（N0）、Grdae2a。
2016/09/21
術後無再発期間（DFI）：2年4ヶ月。
再発乳癌（T2N1M0）と診断。同側SC（鎖骨上）リンパ節転移。
日時不明
再発後の化学療法施行後、進行（PD）と診断。
2018/08/20
BRCA遺伝子変異状況：BRCA1のみ陽性（遺伝カウンセリング：未）。
MIB-1：44％、p53：90％、HER2：陰性、ER：0％（陰性）、PgR：0％（陰性）。
2018/11/22
病変部位：軟部組織（リンバ節）。PS：0。
リムパーザ錠投与開始。
日時不明
部分奏効（PR）と診断。
2018/11/26
この数日嘔吐で食事が取れない時があった。処方開始後1週間で外来に来られて発覚。外来処置で点滴を行った。
2018/12/20
Grade3未満の白血球減少症発現。
2019/03/29
Grade3未満の貧血発現。
日時不明
投与後11ヶ月経過：ほぼ完全奏効（CR）。
2019/11/11
病変の悪化なし。



喫煙習慣：不明。
合併症、併用療法の実施状況（薬物治療以外）：なし。
"""

# 行単位で句切る
text_list = sample_text.split('\n')

In [19]:
tic = time.time()
result_df=inference.pred_pick(text_list)
reftime = time.time() - tic
print(reftime)

0.31621241569519043


**cpu実行速度**

In [15]:
# cpu
tic = time.time()
inference.pred_pick(['胸水は徐々に増加していき、呼吸困難あり。胸腔穿刺ドレナージ(1600ML)実施。']*100)
reftime = time.time() - tic
print(reftime)

9.064430236816406


In [101]:
sample_text ="""

2014/06/13
Bp＋Ax（乳房部分切除＋腋窩リンパ節郭清）（N0）、Grdae2a。
2016/09/21
術後無再発期間（DFI）：2年4ヶ月。
再発乳癌（T2N1M0）と診断。同側SC（鎖骨上）リンパ節転移。
日時不明
再発後の化学療法施行後、進行（PD）と診断。
2018/08/20
BRCA遺伝子変異状況：BRCA1のみ陽性（遺伝カウンセリング：未）。
MIB-1：44％、p53：90％、HER2：陰性、ER：0％（陰性）、PgR：0％（陰性）。
2018/11/22
病変部位：軟部組織（リンバ節）。PS：0。
リムパーザ錠投与開始。
日時不明
部分奏効（PR）と診断。
2018/11/26
この数日嘔吐で食事が取れない時があった。処方開始後1週間で外来に来られて発覚。外来処置で点滴を行った。
2018/12/20
Grade3未満の白血球減少症発現。
2019/03/29
Grade3未満の貧血発現。
日時不明
投与後11ヶ月経過：ほぼ完全奏効（CR）。
2019/11/11
病変の悪化なし。



喫煙習慣：不明。
合併症、併用療法の実施状況（薬物治療以外）：なし。
"""

# 行単位で句切る
text_list = sample_text.split('\n')

In [108]:
tic = time.time()
result_df=inference.pred_pick(text_list)
reftime = time.time() - tic
print(reftime)

5.129703044891357


In [117]:
result_df

,row,pred_pick
0,,[]
1,,[]
2,2014/06/13,[]
3,Bp＋Ax（乳房部分切除＋腋窩リンパ節郭清）（N0）、Grdae2a。,[乳房]
4,2016/09/21,[]
5,術後無再発期間（DFI）：2年4ヶ月。,[]
6,再発乳癌（T2N1M0）と診断。同側SC（鎖骨上）リンパ節転移。,"[再発乳癌, リンパ節転移]"
7,日時不明,[]
8,再発後の化学療法施行後、進行（PD）と診断。,[再発]
9,2018/08/20,[]
